# 고객요청의도 분석하기 (하수도 관련 문의 - 요금, 검침)

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf unstructured sentence-transformers langchain-community

# 데이터 읽어오기

In [ ]:
import pandas as pd

In [ ]:
json_file_path = '민원(콜센터) 질의응답_다산콜센터_생활하수도 관련 문의_Training.json'
df = pd.read_json(json_file_path)
df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,다산콜센터,생활하수도 관련 문의,B2355,고객,1,계량기고장,,Q,수도 계량기가 고장난 것 같습니다.,,,,"계량기, 고장",계량기/수도계량기/ 고장/고장,고장
1,다산콜센터,생활하수도 관련 문의,B2355,상담사,2,,계량기고장,Q,,증상이 어떻게 됩니까?,,,증상,증상/증상,증상
2,다산콜센터,생활하수도 관련 문의,B2355,고객,3,계량기고장,,A,,,침이 회전을 안합니다.,,"침, 회전",침/수도계량기/ 회전/수도계량기,"회전,수도계량기"
3,다산콜센터,생활하수도 관련 문의,B2355,상담사,4,,계량기고장,A,,,,고장 발생 시 교체를 위하여 민원 신고 및 자체 검침을 통해 교체해주셔야 합니다.,"고장, 교체, 민원 신고, 자체 검침",고장/고장/ 교체/행정서비스/ 민원 신고/민원/ 자체검침/검침유형,"교체,검침유형"
4,다산콜센터,생활하수도 관련 문의,B2355,고객,5,계량기고장,,Q,민원 신고는 어디서 합니까?,,,,민원 신고,민원 신고/민원,"민원신고,민원"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40612,다산콜센터,생활하수도 관련 문의,B36388,상담사,17,,요금납부,Q,,추가로 궁금하신점 있으십니까?,,,,,
40613,다산콜센터,생활하수도 관련 문의,B36388,고객,18,요금납부,,A,,,인터넷납부로 진행할게요.,,인터넷납부,인터넷납부/요금납부,"인터넷납부,요금납부"
40614,다산콜센터,생활하수도 관련 문의,B36388,상담사,19,,요금납부,A,,,,서울시ETAX시스템(http://etax.seoul.go.kr) 또는 각 은행 인터...,,,
40615,다산콜센터,생활하수도 관련 문의,B36388,상담사,20,,요금납부,Q,,추가로 궁금하신점 있으십니까?,,,,,


In [ ]:
len(df)

40617

In [ ]:
df.columns

Index(['도메인', '카테고리', '대화셋일련번호', '화자', '문장번호', '고객의도', '상담사의도', 'QA',
       '고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변', '개체명 ', '용어사전', '지식베이스'],
      dtype='object')

# 각 column별 unique한 값 확인하기 (생략)

In [ ]:
고객의도_리스트 = df['고객의도'].unique()
for idx, 고객의도 in enumerate(고객의도_리스트):
  print(idx, 고객의도)

# 고객의도가 '요금'과 관련된 데이터만 필터링하기

In [ ]:
요금_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '요금' in 의도]
요금_키워드_포함_고객의도_리스트

In [ ]:
len(요금_키워드_포함_고객의도_리스트)

100

In [ ]:
filtered_df = df[(df['고객의도'] == '수도요금조회') & (df['문장번호'] == 1)]
filtered_df

In [ ]:
filtered_df = df[(df['고객의도'] == '수도요금납부') & (df['문장번호'] == 1)]
filtered_df

In [ ]:
filtered_df = df[(df['고객의도'] == '정산요금확인') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
140,다산콜센터,생활하수도 관련 문의,B2362,고객,1,정산요금확인,,Q,이사 정산 요금을 좀 확인하려고 합니다.,,,,정산 요금,정산 요금/요금,"정산요금,요금"
3220,다산콜센터,생활하수도 관련 문의,B3455,고객,1,정산요금확인,,Q,이사 정산 요금을 좀 확인하려고 합니다.,,,,정산 요금,정산 요금/요금,"정산요금,요금"
4340,다산콜센터,생활하수도 관련 문의,B3598,고객,1,정산요금확인,,Q,수고하십니다. 정산요금 확인 좀 하고 싶어서요,,,,"수고, 정산요금, 확인",수고/노고/ 확인/점검,"정산요금,점검"
10520,다산콜센터,생활하수도 관련 문의,B16599,고객,1,정산요금확인,,Q,이사 정산 요금 확인해보고 싶어요.,,,,정산 요금,정산 요금/요금,"정산요금,요금"
12918,다산콜센터,생활하수도 관련 문의,B22593,고객,1,정산요금확인,,Q,이사 정산 요금을 좀 확인하려는데요.,,,,정산 요금,정산 요금/요금,"정산요금,요금"
17342,다산콜센터,생활하수도 관련 문의,B23317,고객,1,정산요금확인,,Q,이사요금 정산을 했는데 확인 좀 하고 싶어서요.,,,,이사요금,이사요금/요금/비용/금액,"이사요금,금액"
19471,다산콜센터,생활하수도 관련 문의,B23450,고객,1,정산요금확인,,Q,이사합니다. 수도요금 정산하고 싶어요.,,,,"이사, 정산, 요금, 확인","정산/정리, 요금/요금","정산,요금"


# 고객의도가 '검침'과 관련된 데이터만 필터링하기

In [ ]:
검침_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '검침' in 의도]
검침_키워드_포함_고객의도_리스트

['자가 수도검침',
 '수도검침',
 '수도검침문의',
 '검침',
 '수도자가검침',
 '검침원 정보',
 '검침문의',
 '자가검침',
 '검침신청',
 '자가검침 변경',
 '수도계량기 원격검침',
 '계량기검침이상, 누수',
 '수도요금 자가검침',
 '수도요금 검침 ',
 '계량기 검침',
 '수도계량기 검침 ']

In [ ]:
len(검침_키워드_포함_고객의도_리스트)

16

In [ ]:
filtered_df = df[(df['고객의도'] == '자가 수도검침') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
1500,다산콜센터,생활하수도 관련 문의,B2518,고객,1,자가 수도검침,,Q,수도검침을 본인이하면 요금 감면이 되나요?,,,,"수도검침, 본인, 요금, 감면",요금/돈/ 감면/면제,"본인,면제"
1520,다산콜센터,생활하수도 관련 문의,B2519,고객,1,자가 수도검침,,Q,안녕하세요. 수도검침 문의드려요.,,,,수도검침,수도검침/검침,"수도검침,검침"
29574,다산콜센터,생활하수도 관련 문의,B25496,고객,1,자가 수도검침,,Q,수도검침을 본인이하면 요금 감면이 되는건가요?,,,,"수도검침, 본인, 요금, 감면","요금/돈, 감면/면제","본인,면제"
29594,다산콜센터,생활하수도 관련 문의,B25497,고객,1,자가 수도검침,,Q,안녕하세요. 수도검침 문의드립니다.,,,,수도검침,수도검침/검침,"수도검침,검침"
33757,다산콜센터,생활하수도 관련 문의,B26135,고객,1,자가 수도검침,,Q,수도검침을 본인이하면 요금 감면이 되는건가요?,,,,"수도검침, 본인, 요금, 감면","요금/돈, 감면/면제","본인,면제"
33777,다산콜센터,생활하수도 관련 문의,B26136,고객,1,자가 수도검침,,Q,안녕하세요. 수도검침 문의드립니다.,,,,수도검침,수도검침/검침,"수도검침,검침"


In [ ]:
filtered_df = df[(df['고객의도'] == '수도검침') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
2931,다산콜센터,생활하수도 관련 문의,B2939,고객,1,수도검침,,Q,저희집 계량기가 오래된것 같은데 교체는 어디에서 해주나요?,,,,"계량기,교체",,교체
9560,다산콜센터,생활하수도 관련 문의,B11358,고객,1,수도검침,,Q,안녕하세요. 저희집 수도 검침일은 언제인지 알수 있나요?,,,,"집, 검침일","집/거주지, 검침일/날짜","검침일,날짜"
10880,다산콜센터,생활하수도 관련 문의,B16741,고객,1,수도검침,,Q,우리집 수도 검침일이 궁금한데요?,,,,"집, 수도, 검침일","집/거주지, 검침일/날짜","수도,날짜"
11300,다산콜센터,생활하수도 관련 문의,B16762,고객,1,수도검침,,Q,우리 집 수도 검침 언제 하는지 궁금하네요?,,,,"집, 수도검침, 언제, 궁금","집/거주지, 수도검침/검침","수도검침,검침"
21487,다산콜센터,생활하수도 관련 문의,B23666,고객,1,수도검침,,Q,우리집 수도는 언제 검침하나요?,,,,"우리집, 수도, 언제, 검침","수도, 언제/시기, 검침/검침","수도,검침"
25971,다산콜센터,생활하수도 관련 문의,B24421,고객,1,수도검침,,Q,저희집 수도 검침일은 언제인가요?,,,,"집, 수도, 검침일","집/거주지, 검침일/날짜","수도,날짜"


In [ ]:
filtered_df = df[(df['고객의도'] == '검침') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
5917,다산콜센터,생활하수도 관련 문의,B4000,고객,1,검침,,Q,수도요금 검침 언제 하나요?,,,,"집, 수도검침, 언제, 궁금",집/거주지/ 수도검침/검침,"수도검침,검침"
12540,다산콜센터,생활하수도 관련 문의,B22356,고객,1,검침,,Q,우리 집 수도 검침 언제 하는지 궁금해요.,,,,"집, 수도검침, 언제, 궁금","집/거주지, 수도검침/검침","수도검침,검침"
39430,다산콜센터,생활하수도 관련 문의,B35996,고객,1,검침,,Q,우리 집 수도 검침 언제 하는지 궁금해요.,,,,"집, 수도검침, 언제, 궁금","집/거주지, 수도검침/검침","수도검침,검침"


# 사용자 질문으로부터 '고객요청의도'추출하기

# OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "your_OpenAI_key"

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature = 0, openai_api_key = OPENAI_KEY)

# create_tagging_chain으로 고객 요청의도 추출하기

In [ ]:
from langchain.chains import create_tagging_chain

schema = {
    "properties": {
        "고객요청의도": {
            "type" : "string",
            "enum": ['수도요금',
                     '검침'
                     ]
        }
    }
}
extraction_chain = create_tagging_chain(schema, llm)

In [ ]:
chain_run_result = chain.run("수도요금 문의하고 싶습니다.")
chain_run_result

{'고객요청의도': '수도요금'}

# Vectorstore에 저장하기 위한 데이터 정제하기

In [ ]:
요금_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '요금' in 의도]
요금_키워드_포함_고객의도_리스트

In [ ]:
검침_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '검침' in 의도]
검침_키워드_포함_고객의도_리스트

In [ ]:
최종_필터링용_고객의도_리스트 = 요금_키워드_포함_고객의도_리스트 + 검침_키워드_포함_고객의도_리스트
최종_필터링용_고객의도_리스트

In [ ]:
len(최종_필터링용_고객의도_리스트)

116

In [ ]:
filtered_df = df[df['고객의도'].isin(최종_필터링용_고객의도_리스트)]
filtered_df

# 중복 데이터 제거

In [ ]:
duplicates = filtered_df.duplicated(subset=filtered_df.columns.difference(['대화셋일련번호']), keep=False)

df_cleaned = filtered_df[~duplicates]
df_cleaned

In [ ]:
대화셋_set = set()
for count, (index, row) in enumerate(df_cleaned.iterrows()):
  print(count, row['대화셋일련번호'], row['고객질문(요청)'])
  대화셋_set.add(row['대화셋일련번호'])

In [ ]:
filtered_df = df[df['대화셋일련번호'].isin(대화셋_set)]
filtered_df

In [ ]:
def combine_columns(group):
  sorted_group = group.sort_index()
  combined_string = " ".join(
      sorted_group[['고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변']].astype(str).values.flatten()
  )

  customer_intent = sorted_group['고객의도'].iloc[0]

  if pd.isna(customer_intent) or customer_intent == '':
    customer_intent = sorted_group['상담사의도'].iloc[0]

  return pd.Series({'combined_string': combined_string, '고객의도': customer_intent})

In [ ]:
combined_result = filtered_df.groupby('대화셋일련번호').apply(combine_columns).reset_index()
combined_result

In [ ]:
combined_result[combined_result['대화셋일련번호']=='B10715']['combined_string'].values[0]

'이사를 가는데 수도요금도 정산해야할것같은데요.어떻게하죠?       네.고객님..먼저 개인정보 여쭐께요    현재 거주하는곳 주소 부탁드립니다   대전시 OOO OOO 000-00 에요   네.가입자분 성함은 어떻게되시나요?     000이에요     네.확인되셨구요.   계량기 검침판 숫자 확인가능하실까요?   검침판 숫자요? 잠시만요       네.천천히 알려주세요 큰숫자 알려드리면 되나요? 13이라고 있는데       잠시만요. 요금은 27960원이시네요 이거 어디로 보내야해요?       계좌번호 알려드릴테니 입금하시면 되세요   불러주세요     OOOOOO-OOOOOOOOO-OOO 이구요. OO은행입니다 오늘중으로 입금하면 되죠?       네.입금하실때 고객님성함으로 하셔야 확인되십니다.   알겠습니다.수고하세요     좋은하루되세요'

In [ ]:
for index, row in combined_result.iterrows():
  print(f"대화셋일련번호: {row['대화셋일련번호']}, 고객요청의도: {row['고객의도']}, Combined String: {row['combined_string']}")

# Document 생성 및 Vectorstore에 Embedding해서 저장

In [ ]:
from langchain.schema import Document

docs = []
수도요금_docs = []
검침_docs = []

# 각 row의 값들을 순회하면서 출력
for index, row in combined_result.iterrows():

  doc = Document(page_content = row['combined_string'], metadata={'대화셋일련번호': row['대화셋일련번호'], '고객요청의도': row['고객의도']})
  docs.append(doc)

  if row['고객의도'] == '수도요금':
    수도요금_docs.append(doc)
  elif row['고객의도'] == '검침':
    검침_docs.append(doc)

In [ ]:
docs

In [ ]:
수도요금_docs

In [ ]:
검침_docs

[Document(metadata={'대화셋일련번호': 'B4000', '고객요청의도': '검침'}, page_content='수도요금 검침 언제 하나요?       서울시 전지역을 홀수달, 짝수달로 나누어 2달에 한번 검침원이 직접 검침을 하고 있습니다. 며칠에 하는 지는 모르고요?        전월 21일부터 당월 8일까지로 하고 있습니다.  정확한 날짜는요?        수도요금 고지서 3번째 쪽 상단에 있습니다.  1일부터 말일까지 쓴 금액이 나오는 건가요?        검침일 기준으로 부과됩니다.  자가검침 하는 거 있던데 그건 어떻게 해요?        수도계량기 지침을 확인하여 인터넷 또는 ARS를 통해 지침을 입력하시면 됩니다.  혜택있어요?        1회당 600원 할인입니다. 그거 잘 하고 있는지 확인돼요?        네 관리되고 있습니다.  어떻게요?        2회 이상 미검침 시 직권해지, 착오입력 등 방지위해 1년에 1회 정도 확인검침을 실시합니다.    그렇군요.    더 궁금한 점 있으신가요?     아니요.     네 즐거운 하루되세요.')]

# Embedding

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sroberta-multitask" #(KorNLU 데이터넷에 학습시킨 한국어 임베딩 모델)
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_model, collection_name="all")

In [ ]:
수도요금_vectorstore = Chroma.from_documents(documents=수도요금_docs, embedding=embedding_model, collection_name="waterbill")

In [ ]:
검침_vectorstore = Chroma.from_documents(documents=검침_docs, embedding=embedding_model, collection_name="meterreading")

In [ ]:
len(vectorstore)
len(수도요금_vectorstore)
len(검침_vectorstore)

1

# Sanity Check

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
수도요금_retriever = 수도요금_vectorstore.as_retriever(search_kwargs={"k": 1})
검침_retriever = 검침_vectorstore.as_retriever(search_kwargs={"k": 1})

In [ ]:
retrieved_docs = retriever.invoke(
    "다자녀는 수도_요금 할인되나요?"
)
print(retrieved_docs[0].page_content)

다자녀 할인 뭐가 있다고 하던데요.     다자녀 할인 혜택에 대해서 궁금하신가요?     네 맞아요.  어떤 할인 혜택이 있나요?        다자녀 가구를 대상으로 전기세, 수도세, 도시가스 할인을 받으실 수 있습니다.  수도요금 할인 받으려면 어떻게 해야 하나요?       상수도 요금 감면 신청서를 작성, 제출해주시면 됩니다.  필요서류는 없나요?       주민등록 등본이 추가로 필요합니다. 수도요금 할인에 등본이 필요한가요?       다자녀 가구 확인이 필요하기 때문입니다.   아 그렇겠네요.  지금 신청하면 다음달 부터 바로 할인 되나요?       매달 25일 이전에 신청을 완료하시면 다음달 부터 할인 적용이 완료되어 그 다음달 할인내역 확인이 가능합니다. 25일 이전에 신청하면 다다음달에 확인이 된다는 거죠?       네 그렇습니다. 혹시 전기세도 여기서 신청하면 되나요?       전기세는 한전으로 직접 신청하셔야 합니다.   아 그렇군요. 자세한 안내 고마워요     네 감사합니다. 


In [ ]:
retrieved_docs = 수도요금_retriever.invoke(
    "다자녀는 수도_요금 할인되나요?"
)
print(retrieved_docs[0].page_content)

수도요금 좀 계산해 주시겠어요?     네, 계량기구경과 사용량 알려주시면 말씀드리겠습니다.     계량기구경은 20mm고, 사용량은 52톤 되는거 같습니다.     그러시면, 68660원으로 계산되세요.  저희 집이 할인 혜택이 몇 개 있는데, 반영해 주시겠어요?     네, 어떤 혜택을 받는지 불러주세요.     독립유공자, 다자녀 하수도, 자가검침하고, 전자고지 입니다.     감면혜택을 반영하면 49710원입니다. 독립유공자 감면혜택은 얼마나 되나요?       독립유공자 감면혜택은 9300원 입니다. 다자녀는요?       8830원 입니다. 자가검침은 얼마나 할인 받나요?       600원입니다. 전자고지는요?       220원 감면 받으세요. 전체 18950원 할인 되는게 맞나요?       네, 맞습니다.   네, 알겠습니다. 감사합니다.     도움이 되셨길 바랍니다. 


In [ ]:
retrieved_docs = 검침_retriever.invoke(
    "다자녀는 수도_요금 할인되나요?"
)
print(retrieved_docs[0].page_content)

수도요금 검침 언제 하나요?       서울시 전지역을 홀수달, 짝수달로 나누어 2달에 한번 검침원이 직접 검침을 하고 있습니다. 며칠에 하는 지는 모르고요?        전월 21일부터 당월 8일까지로 하고 있습니다.  정확한 날짜는요?        수도요금 고지서 3번째 쪽 상단에 있습니다.  1일부터 말일까지 쓴 금액이 나오는 건가요?        검침일 기준으로 부과됩니다.  자가검침 하는 거 있던데 그건 어떻게 해요?        수도계량기 지침을 확인하여 인터넷 또는 ARS를 통해 지침을 입력하시면 됩니다.  혜택있어요?        1회당 600원 할인입니다. 그거 잘 하고 있는지 확인돼요?        네 관리되고 있습니다.  어떻게요?        2회 이상 미검침 시 직권해지, 착오입력 등 방지위해 1년에 1회 정도 확인검침을 실시합니다.    그렇군요.    더 궁금한 점 있으신가요?     아니요.     네 즐거운 하루되세요.


# (중요!) Multiple Retriever를 활용하여 연관된 문서 찾아오기

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_KEY)

In [ ]:
retrievers = {
    "" : retriever,
    "수도요금": 수도요금_retriever,
    "검침": 검침_retriever
}

In [ ]:
from langchain_core.runnables import chain

In [ ]:
@chain
def custom_chain(question):
  response = extraction_chain.run(question)
  retriever = retrievers[response.get('고객의도')or""]
  return retriever.invoke(question)

In [ ]:
custom_chain.invoke("다자녀는 수도 요금 할인되나요?")

[Document(metadata={'고객요청의도': '요금할인 ', '대화셋일련번호': 'B3577'}, page_content='다자녀 할인 뭐가 있다고 하던데요.     다자녀 할인 혜택에 대해서 궁금하신가요?     네 맞아요.  어떤 할인 혜택이 있나요?        다자녀 가구를 대상으로 전기세, 수도세, 도시가스 할인을 받으실 수 있습니다.  수도요금 할인 받으려면 어떻게 해야 하나요?       상수도 요금 감면 신청서를 작성, 제출해주시면 됩니다.  필요서류는 없나요?       주민등록 등본이 추가로 필요합니다. 수도요금 할인에 등본이 필요한가요?       다자녀 가구 확인이 필요하기 때문입니다.   아 그렇겠네요.  지금 신청하면 다음달 부터 바로 할인 되나요?       매달 25일 이전에 신청을 완료하시면 다음달 부터 할인 적용이 완료되어 그 다음달 할인내역 확인이 가능합니다. 25일 이전에 신청하면 다다음달에 확인이 된다는 거죠?       네 그렇습니다. 혹시 전기세도 여기서 신청하면 되나요?       전기세는 한전으로 직접 신청하셔야 합니다.   아 그렇군요. 자세한 안내 고마워요     네 감사합니다. ')]

In [ ]:
custom_chain.invoke("수도도 자가검침이 가능한가요?")

[Document(metadata={'고객요청의도': '수도자가검침', '대화셋일련번호': 'B4035'}, page_content='수도도 자가검침이 가능한가요?       네. 자가검침이 가능합니다. 아~~자가검침도 가능하군요?       네 그렇습니다. 저희집은 다세대인데 가능한가요?     수도 계량기가 따로 설치되어 있습니까?     네.. 개별로 설치되어 있어요.     네. .가능합니다. 자가검침은 언제 하나요?       지자체 별로 검침 날짜는 차이가 있어서 신청 하시면 따로 안내될 예정입니다. 딱 정해진 날짜에 해야 하나요?       그렇진 않고 5일 기간 내에 하시면 됩니다. 까먹으면 어떡해요?       검침일 2일 전에 이메일이나 메시지로 통보됩니다.   잊어버릴 염려는 없겠네요..     네. 어떻게 신청하나요?       인터넷, 전화, 모바일 아리수 앱에서 가능합니다. 전화번호 부탁드려요.       OOOO-OOOO입니다.')]

In [ ]:
custom_chain.invoke("할인도 되나요?")

[Document(metadata={'고객요청의도': '요금감면제도', '대화셋일련번호': 'B23276'}, page_content='수도요금 할인 납부 가능한가요?     어떤 방법으로 납부하실 건가요?     현금이 싸면 현금이요.     카드, 현금납부 금액은 모두 같습니다. 현금 할인도 없나요? 수도요금 좀 싸게 낼 수 있는 법 없어요?       종이가 아닌 전자고지 신청하시거나 자동납부청구서를 이메일로 받으시면 할인 받으실 수 있습니다. 나이 든 사람이 이메일이 어디 있나요. 다른 법은 없어요?       바로알림서비스를 신청하셔서 문자로 받으시면 할인 받으실 수 있습니다. 바로알림이 뭔가요?       고객님의 휴대폰번호로 수도요금을 SMS로 안내해드리는 겁니다. 집전화로는 안되나요?       SMS 수신이 가능한 집전화라면 가능합니다. 아 그런건 잘 모르겠고 그거 신청하는건 공짜요?       네. 무료입니다. 바로알림 신청하는 방법은요?       서울시 상수도사업본부 온라인민원신청에서 바로알림을 검색하셔서 신청하시거나 전화로도 신청 가능합니다. 얼마나 할인되나요?       상수도요금의 1%를 할인해 드립니다. 수도요금 보통 22000원 정도 나오는데 1%할인이면 얼마인가요?       22000원에서 1%할인 해드리면 220원 감면되고 납부하시게 될 금액은 21780원입니다.')]

# 대화 history 저장을 위한 ConversationBufferMemory 설정

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

<ipython-input-79-3f477d3d00ff>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [ ]:
memory.load_memory_variables({})

{'history': ''}

In [ ]:
def load_memory(input):
  return memory.load_memory_variables({})["history"]

# Tracking을 위한 LangSmith 설정(Optional)

In [ ]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"User Intent Analysis (sewer) (validation test) - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "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"

In [ ]:
unique_id

'c402ea37'

In [ ]:
memory.load_memory_variables({})["history"]

''

In [ ]:
def using_intent_get_relevant_document(user_question):
  history_combined_question = memory.load_memory_variables({})["history"] + '\nHuman::' + user_question

  relevant_document = custom_chain.invoke(history_combined_question)[0]

  return relevant_document

In [ ]:
relevant_document = using_intent_get_relevant_document("다자녀는 수도 요금 할인되나요?")
relevant_document

Document(metadata={'고객요청의도': '요금할인 ', '대화셋일련번호': 'B3577'}, page_content='다자녀 할인 뭐가 있다고 하던데요.     다자녀 할인 혜택에 대해서 궁금하신가요?     네 맞아요.  어떤 할인 혜택이 있나요?        다자녀 가구를 대상으로 전기세, 수도세, 도시가스 할인을 받으실 수 있습니다.  수도요금 할인 받으려면 어떻게 해야 하나요?       상수도 요금 감면 신청서를 작성, 제출해주시면 됩니다.  필요서류는 없나요?       주민등록 등본이 추가로 필요합니다. 수도요금 할인에 등본이 필요한가요?       다자녀 가구 확인이 필요하기 때문입니다.   아 그렇겠네요.  지금 신청하면 다음달 부터 바로 할인 되나요?       매달 25일 이전에 신청을 완료하시면 다음달 부터 할인 적용이 완료되어 그 다음달 할인내역 확인이 가능합니다. 25일 이전에 신청하면 다다음달에 확인이 된다는 거죠?       네 그렇습니다. 혹시 전기세도 여기서 신청하면 되나요?       전기세는 한전으로 직접 신청하셔야 합니다.   아 그렇군요. 자세한 안내 고마워요     네 감사합니다. ')

# 최종 Prompt 및 Chain 생성

In [ ]:
from langchain.prompts import PromptTemplate

template = """다음과 같은 맥락과 채팅히스토리를 사용하여 마지막 질문에 대답하십시오.
맥락: {context}
채팅히스토리: {history}
질문: {question}
도움이 되는 답변:"""

rag_prompt_custom = PromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import MessagesPlaceholder

rag_chain = {"context": using_intent_get_relevant_document, "question": RunnablePassthrough(), "history" : load_memory} | rag_prompt_custom | llm

In [ ]:
def get_result_and_save_memory(user_question):
  chat_result = rag_chain.invoke(user_question)

  memory.chat_memory.add_user_message(user_question)
  memory.chat_memory.add_ai_message(chat_result.content)

  return chat_result.content

# 수도요금, 검침 AI 고객센터 챗봇 성능 evaluation (정성적 평가) - Validation 데이터에 테스트

In [ ]:
json_file_path = '민원(콜센터) 질의응답_다산콜센터_생활하수도 관련 문의_Validation.json'
validation_df = pd.read_json(json_file_path)
validation_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,다산콜센터,생활하수도 관련 문의,B36389,고객,1,저수조청소,,Q,저수조청소가 뭐죠?,,,,저수조청소,저수조청소/저수조청소,저수조청소
1,다산콜센터,생활하수도 관련 문의,B36389,상담사,2,,저수조청소,A,,,,저수조(물탱크)를 깨끗이 청소하여야 깨끗한 수돗물을 드실 수 있으며 청소 등의 위생...,,,
2,다산콜센터,생활하수도 관련 문의,B36389,상담사,3,,저수조청소,Q,,추가로 궁금하신점 있으십니까?,,,,,
3,다산콜센터,생활하수도 관련 문의,B36389,고객,4,저수조청소,,Q,제가 지금 연대앞 00동에서 임대사업을 하는데 여기까지 물이 어떻게오죠?,,,,,,
4,다산콜센터,생활하수도 관련 문의,B36389,상담사,5,,저수조청소,A,,,,아리수 공급경로1. 00취수장2. 00송수3. 00[지역배수지]4. 00[공원]의 ...,공급경로,공급경로/공급경로,공급경로
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083,다산콜센터,생활하수도 관련 문의,B41492,상담사,16,,상수도,A,,,,네. 세균제거하면 경감될수 있습니다.,"세균,경감",세균/세균/경감/경감,경감
5084,다산콜센터,생활하수도 관련 문의,B41492,고객,17,상수도,,Q,수돗물하고는 관련이 없는건가요?,,,,수돗물,수돗물/물,"수돗물,물"
5085,다산콜센터,생활하수도 관련 문의,B41492,상담사,18,,상수도,A,,,,네. 그렇습니다.,,,
5086,다산콜센터,생활하수도 관련 문의,B41492,고객,19,상수도,,Q,수돗물 그냥 먹어도 되는건가요?,,,,수돗물,수돗물/물,"수돗물,물"


In [ ]:
validation_고객의도_리스트 = validation_df['고객의도'].unique()
for idx, 고객의도 in enumerate(validation_고객의도_리스트):
  print(idx, 고객의도)

In [ ]:
요금_키워드_포함_고객의도_리스트 = [의도 for 의도 in validation_고객의도_리스트 if '요금' in 의도]
요금_키워드_포함_고객의도_리스트

In [ ]:
검침_키워드_포함_고객의도_리스트 = [의도 for 의도 in validation_고객의도_리스트 if '검침' in 의도]
검침_키워드_포함_고객의도_리스트

['검침원 정보', '검침문의', '검침']

In [ ]:
최종_필터링용_고객의도_리스트 = 요금_키워드_포함_고객의도_리스트 + 검침_키워드_포함_고객의도_리스트
최종_필터링용_고객의도_리스트

In [ ]:
len(최종_필터링용_고객의도_리스트)

32

In [ ]:
filtered_df = validation_df[validation_df['고객의도'].isin(최종_필터링용_고객의도_리스트)]
filtered_df

# 중복 데이터 제거

In [ ]:
duplicates = filtered_df.duplicated(subset=filtered_df.columns.difference(['대화셋일련번호']), keep=False)

df_cleaned = filtered_df[~duplicates]
df_cleaned

In [ ]:
대화셋_set = set()
for count, (index, row) in enumerate(df_cleaned.iterrows()):
  print(count, row['대화셋일련번호'], row['고객질문(요청)'])
  대화셋_set.add(row['대화셋일련번호'])

In [ ]:
len(대화셋_set)

91

# Case 1 - 검침

In [ ]:
memory.clear()

In [ ]:
temp_df = validation_df[validation_df['대화셋일련번호'] == 'B41393']
temp_df

In [ ]:
llm_response = get_result_and_save_memory("자가검침 관련해서 여쭙고 싶습니다.")
llm_response

'자가검침에 대해 궁금하신 점이 있으시군요. 자가검침일에 검침을 하셨다면, 아리수 홈페이지나 모바일 아리수 앱을 통해 검침 결과를 입력하실 수 있습니다. 자가검침을 신청하셨더라도, 착오 입력 등을 방지하기 위해 1년에 한 번 정도 확인 검침이 이루어지며, 이때는 문자로 점검 안내 일정을 통지해드립니다. 만약 검침을 깜빡하셨다면, 문자로 검침일에 대해 안내를 드리며, 2회 이상 미검침 시에는 직원 해지가 될 수 있으니 주의하시기 바랍니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요.'

In [ ]:
llm_response = get_result_and_save_memory("수도요금 할인은 무슨 일인가요?")
llm_response

'수도요금 할인은 특정 조건을 충족하는 경우에 수도 요금을 감면받을 수 있는 제도입니다. 일반적으로 수도요금 할인은 저소득층, 다자녀 가구, 장애인 가구, 국가유공자 등 사회적 배려 대상자에게 제공됩니다. 각 지역의 수도 사업자마다 할인 조건과 신청 방법이 다를 수 있으므로, 해당 지역의 수도 사업자나 관련 기관에 문의하여 자세한 정보를 확인하시는 것이 좋습니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요.'

In [ ]:
memory.load_memory_variables({})["history"]

'Human: 자가검침 관련해서 여쭙고 싶습니다.\nAI: 자가검침에 대해 궁금하신 점이 있으시군요. 자가검침일에 검침을 하셨다면, 아리수 홈페이지나 모바일 아리수 앱을 통해 검침 결과를 입력하실 수 있습니다. 자가검침을 신청하셨더라도, 착오 입력 등을 방지하기 위해 1년에 한 번 정도 확인 검침이 이루어지며, 이때는 문자로 점검 안내 일정을 통지해드립니다. 만약 검침을 깜빡하셨다면, 문자로 검침일에 대해 안내를 드리며, 2회 이상 미검침 시에는 직원 해지가 될 수 있으니 주의하시기 바랍니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요.\nHuman: 수도요금 할인은 무슨 일인가요?\nAI: 수도요금 할인은 특정 조건을 충족하는 경우에 수도 요금을 감면받을 수 있는 제도입니다. 일반적으로 수도요금 할인은 저소득층, 다자녀 가구, 장애인 가구, 국가유공자 등 사회적 배려 대상자에게 제공됩니다. 각 지역의 수도 사업자마다 할인 조건과 신청 방법이 다를 수 있으므로, 해당 지역의 수도 사업자나 관련 기관에 문의하여 자세한 정보를 확인하시는 것이 좋습니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요.'

In [ ]:
llm_response = get_result_and_save_memory("아 그럼 저도 신청해서 할 수 있는 건가요?")
llm_response

'수도요금 할인을 신청할 수 있는지는 귀하의 상황과 해당 지역의 수도 사업자가 제공하는 할인 조건에 따라 다릅니다. 일반적으로 저소득층, 다자녀 가구, 장애인 가구, 국가유공자 등이 할인 대상이 될 수 있습니다. 귀하가 이러한 조건에 해당하는지 확인하시고, 해당 지역의 수도 사업자나 관련 기관에 문의하여 신청 가능 여부와 절차를 알아보시는 것이 좋습니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요.'

In [ ]:
memory.load_memory_variables({})["history"]

'Human: 자가검침 관련해서 여쭙고 싶습니다.\nAI: 자가검침에 대해 궁금하신 점이 있으시군요. 자가검침일에 검침을 하셨다면, 아리수 홈페이지나 모바일 아리수 앱을 통해 검침 결과를 입력하실 수 있습니다. 자가검침을 신청하셨더라도, 착오 입력 등을 방지하기 위해 1년에 한 번 정도 확인 검침이 이루어지며, 이때는 문자로 점검 안내 일정을 통지해드립니다. 만약 검침을 깜빡하셨다면, 문자로 검침일에 대해 안내를 드리며, 2회 이상 미검침 시에는 직원 해지가 될 수 있으니 주의하시기 바랍니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요.\nHuman: 수도요금 할인은 무슨 일인가요?\nAI: 수도요금 할인은 특정 조건을 충족하는 경우에 수도 요금을 감면받을 수 있는 제도입니다. 일반적으로 수도요금 할인은 저소득층, 다자녀 가구, 장애인 가구, 국가유공자 등 사회적 배려 대상자에게 제공됩니다. 각 지역의 수도 사업자마다 할인 조건과 신청 방법이 다를 수 있으므로, 해당 지역의 수도 사업자나 관련 기관에 문의하여 자세한 정보를 확인하시는 것이 좋습니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요.\nHuman: 아 그럼 저도 신청해서 할 수 있는 건가요?\nAI: 수도요금 할인을 신청할 수 있는지는 귀하의 상황과 해당 지역의 수도 사업자가 제공하는 할인 조건에 따라 다릅니다. 일반적으로 저소득층, 다자녀 가구, 장애인 가구, 국가유공자 등이 할인 대상이 될 수 있습니다. 귀하가 이러한 조건에 해당하는지 확인하시고, 해당 지역의 수도 사업자나 관련 기관에 문의하여 신청 가능 여부와 절차를 알아보시는 것이 좋습니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요.'

In [ ]:
llm_response = get_result_and_save_memory("신청 하는 방법을 알 수 있을까요?")
llm_response

'수도요금 할인을 신청하는 방법은 해당 지역의 수도 사업자나 관련 기관에 따라 다를 수 있습니다. 일반적으로는 다음과 같은 절차를 따르게 됩니다:\n\n1. **자격 확인**: 먼저, 귀하가 수도요금 할인 대상에 해당하는지 확인해야 합니다. 저소득층, 다자녀 가구, 장애인 가구, 국가유공자 등이 일반적인 대상입니다.\n\n2. **필요 서류 준비**: 자격을 증명할 수 있는 서류를 준비해야 합니다. 예를 들어, 소득 증명서, 가족관계증명서, 장애인 증명서 등이 필요할 수 있습니다.\n\n3. **신청서 작성**: 해당 기관에서 제공하는 신청서를 작성합니다. 이 신청서는 수도 사업자의 웹사이트에서 다운로드하거나, 직접 방문하여 받을 수 있습니다.\n\n4. **신청서 제출**: 작성한 신청서와 준비한 서류를 해당 수도 사업자나 관련 기관에 제출합니다. 온라인으로 제출할 수 있는 경우도 있으니 확인해 보세요.\n\n5. **결과 통보**: 신청이 승인되면, 수도요금 할인 적용 여부와 적용 시기를 통보받게 됩니다.\n\n자세한 사항은 귀하의 지역 수도 사업자나 관련 기관에 직접 문의하여 확인하시기 바랍니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요.'

# Case 2 - 수도요금

In [ ]:
memory.clear()

In [ ]:
temp_df = validation_df[validation_df['대화셋일련번호'] == 'B36668']
temp_df

In [ ]:
llm_response = get_result_and_save_memory("공동 수도요금은 어떻게 계산되는 거에요?")
llm_response

'공동수도가 있는 빌라의 경우, 수도요금은 주계량기와 세대별 계량기가 함께 설치되어 있는 경우에 주계량 사용량에서 세대별 계량기 총사용량을 빼서 공동사용량을 결정합니다. 이 공동사용량을 세대별로 균등하게 분할한 후, 각 세대별 사용량에 합산하여 수도요금을 고지합니다. 따라서, 누군가 많이 사용하면 공동으로 부담하는 요금이 증가할 수 있습니다.'

In [ ]:
memory.load_memory_variables({})["history"]

'Human: 공동 수도요금은 어떻게 계산되는 거에요?\nAI: 공동수도가 있는 빌라의 경우, 수도요금은 주계량기와 세대별 계량기가 함께 설치되어 있는 경우에 주계량 사용량에서 세대별 계량기 총사용량을 빼서 공동사용량을 결정합니다. 이 공동사용량을 세대별로 균등하게 분할한 후, 각 세대별 사용량에 합산하여 수도요금을 고지합니다. 따라서, 누군가 많이 사용하면 공동으로 부담하는 요금이 증가할 수 있습니다.'